# 📚 Guía de Buenas Prácticas en Python Moderno (3.10+)

Este notebook recopila las **mejores prácticas profesionales** de código Python que todo desarrollador debe conocer. Estas prácticas se aplican en todos los proyectos de este repositorio, especialmente en los notebooks de IA y bases de datos.

---

## 🎯 Objetivo Pedagógico

**Python puede ser tan disciplinado y profesional como Java, C# o TypeScript** si seguimos estas reglas. El código resultante es:
- ✅ Más legible y mantenible
- ✅ Más fácil de depurar y testear
- ✅ Más robusto ante errores
- ✅ Más colaborativo (otros desarrolladores lo entienden mejor)

---

## 1️⃣ Type Hints OBLIGATORIOS con Sintaxis Moderna

### ✅ Sintaxis Moderna (Python 3.10+)

Desde Python 3.10, podemos usar el operador `|` para tipos union, que es más legible que `Optional` y `Union`.

In [1]:
# ✅ MODERNO: Usar | en lugar de Optional
nombre: str | None = None
edad: int | None = 42
puntuacion: float | None = 9.5

# Para múltiples tipos
identificador: str | int = "ABC123"  # Puede ser string o int

# ❌ ANTIGUO (aún funciona pero no es lo más moderno)
from typing import Optional, Union
nombre_antiguo: Optional[str] = None
identificador_antiguo: Union[str, int] = "ABC123"

### 📊 Variables Locales con Tipos Explícitos

**SIEMPRE** declara el tipo de las variables, incluso las locales:

In [2]:
# ✅ BIEN: Tipos explícitos
def procesar_datos(datos: dict[str, str]) -> list[str]:
    resultado: list[str] = []  # Tipo explícito
    contador: int = 0
    mensaje: str = "Procesando..."
    
    for clave, valor in datos.items():
        elemento: str = f"{clave}: {valor}"  # Tipo explícito
        resultado.append(elemento)
        contador += 1
    
    return resultado

# ❌ MAL: Sin tipos (difícil de mantener)
def procesar_datos_mal(datos):
    resultado = []  # ¿Qué contiene?
    for k, v in datos.items():
        resultado.append(f"{k}: {v}")
    return resultado

### 📋 Comparación de Sintaxis de Tipos Nullable

| Antiguo (Python 3.5-3.9) | Moderno (Python 3.10+) |
|---------------------------|------------------------|
| `Optional[str]` | `str \| None` |
| `Optional[int]` | `int \| None` |
| `Union[str, int]` | `str \| int` |
| `Optional[List[str]]` | `list[str] \| None` |
| `Optional[Dict[str, int]]` | `dict[str, int] \| None` |

**🎓 Lección:** La nueva sintaxis es más concisa, legible y se alinea con otros lenguajes modernos (TypeScript, Kotlin).

---

## 2️⃣ UN SOLO Return por Función (Máximo 2 Justificados)

### ❌ MAL: Múltiples Returns Confusos

In [3]:
# ❌ MAL: 7 returns que dificultan el seguimiento del flujo
def ejecutar_mal(sql, db):
    if not sql:
        return "error1"
    if db is None:
        return "error2"
    try:
        resultado = db.execute(sql)
        if not resultado:
            return "vacio"
        if len(resultado) > 100:
            return "demasiados_datos"
        return "ok"
    except ConnectionError:
        return "error_conexion"
    except Exception:
        return "error_generico"

### ✅ BIEN: UN Return al Final (con Guard Clauses Permitidas)

In [4]:
# ✅ BIEN: Guard clauses al inicio + UN return al final
def ejecutar_bien(sql: str, db: object | None) -> str:
    """Ejecuta una query SQL y retorna el estado.
    
    Args:
        sql: Consulta SQL a ejecutar
        db: Objeto de conexión a la base de datos
        
    Returns:
        Estado de la ejecución: 'ok', 'error', 'vacio', etc.
    """
    # Guard clauses: validaciones tempranas (máximo 2 returns permitidos)
    if not sql:
        return "error"
    if db is None:
        return "error"
    
    # Lógica principal con variable de resultado
    estado: str = _ejecutar_y_validar(sql, db)
    return estado


def _ejecutar_y_validar(sql: str, db: object) -> str:
    """Función auxiliar privada que ejecuta y valida."""
    estado: str = "error_generico"  # Valor por defecto
    
    try:
        resultado = db.execute(sql)  # type: ignore
        
        if not resultado:
            estado = "vacio"
        elif len(resultado) > 100:
            estado = "demasiados_datos"
        else:
            estado = "ok"
            
    except ConnectionError:
        estado = "error_conexion"
    except Exception:
        estado = "error_generico"
    
    return estado  # UN SOLO return

**🎓 Lección:** Múltiples returns dificultan el debugging, testing y seguimiento del flujo. Usa guard clauses para validaciones tempranas, pero luego UN solo return al final.

---

## 3️⃣ Guard Clauses al Inicio (Fail Fast)

Validar precondiciones **ANTES** de la lógica principal evita anidamiento excesivo y estados inconsistentes.

In [5]:
# ❌ MAL: Anidamiento profundo
def procesar_pedido_mal(pedido, cliente, stock):
    if pedido is not None:
        if cliente is not None:
            if stock > 0:
                # Lógica principal enterrada 3 niveles
                return "procesado"
            else:
                return "sin_stock"
        else:
            return "cliente_invalido"
    else:
        return "pedido_invalido"


# ✅ BIEN: Guard clauses (salidas tempranas)
def procesar_pedido_bien(pedido: dict | None, cliente: dict | None, stock: int) -> str:
    """Procesa un pedido validando precondiciones.
    
    Args:
        pedido: Datos del pedido
        cliente: Datos del cliente
        stock: Cantidad en inventario
        
    Returns:
        Estado del procesamiento
    """
    # Validaciones tempranas (guard clauses)
    if pedido is None:
        return "pedido_invalido"
    
    if cliente is None:
        return "cliente_invalido"
    
    if stock <= 0:
        return "sin_stock"
    
    # Lógica principal sin anidamiento
    resultado: str = _ejecutar_procesamiento(pedido, cliente, stock)
    return resultado


def _ejecutar_procesamiento(pedido: dict, cliente: dict, stock: int) -> str:
    """Lógica de procesamiento (ya validado)."""
    # Aquí va la lógica real
    return "procesado"

**🎓 Lección:** Guard clauses reducen anidamiento, mejoran legibilidad y detectan errores temprano (fail-fast principle).

---

## 4️⃣ Funciones Auxiliares Privadas (Prefijo `_`)

Separa lógica compleja en funciones pequeñas, testables y con prefijo `_` para indicar uso interno.

In [6]:
# ✅ BIEN: Separación de responsabilidades
import json

def procesar_respuesta_llm(raw_output: str) -> dict[str, str]:
    """Procesa la respuesta de un LLM (función pública).
    
    Args:
        raw_output: Respuesta cruda del modelo de lenguaje
        
    Returns:
        Diccionario con datos parseados
    """
    # Lógica principal clara y legible
    texto_limpio: str = _limpiar_markdown(raw_output)
    datos: dict[str, str] = _parsear_json(texto_limpio)
    datos_validados: dict[str, str] = _validar_campos(datos)
    return datos_validados


def _limpiar_markdown(texto: str) -> str:
    """Función privada: Elimina bloques de código Markdown."""
    texto_limpio: str = texto.strip()
    if texto_limpio.startswith("```"):
        lineas: list[str] = texto_limpio.split("\n")
        texto_limpio = "\n".join(lineas[1:-1]).strip()
    return texto_limpio


def _parsear_json(texto: str) -> dict[str, str]:
    """Función privada: Parsea JSON con manejo de errores."""
    try:
        datos: dict[str, str] = json.loads(texto)
        return datos
    except json.JSONDecodeError:
        return {"error": "json_invalido"}


def _validar_campos(datos: dict[str, str]) -> dict[str, str]:
    """Función privada: Valida que existan campos requeridos."""
    if "error" not in datos and "respuesta" in datos:
        return datos
    return {"error": "campos_faltantes"}

**🎓 Lección:** 
- Funciones públicas (sin `_`): API que otros módulos pueden usar
- Funciones privadas (con `_`): Detalles de implementación internos
- Funciones pequeñas (< 30 líneas) son más fáciles de entender y testear

---

## 5️⃣ Evitar Variables Globales en Funciones

Las variables globales dificultan testing, crean efectos secundarios y hacen el código menos predecible.

In [7]:
# ❌ MAL: Uso de globales
DB_CONNECTION = None  # Global

def ejecutar_query_mal(sql: str) -> list:
    global DB_CONNECTION  # Acceso a global
    if DB_CONNECTION is None:
        return []
    return DB_CONNECTION.execute(sql)  # type: ignore


# ✅ BIEN: Pasar como parámetro
def ejecutar_query_bien(sql: str, db_connection: object | None) -> list:
    """Ejecuta una query SQL.
    
    Args:
        sql: Consulta a ejecutar
        db_connection: Conexión a la base de datos
        
    Returns:
        Resultados de la query
    """
    if db_connection is None:
        return []
    return db_connection.execute(sql)  # type: ignore

### 🎯 Patrón para Contexto Compartido

Cuando necesites compartir estado, usa objetos de estado (como `TypedDict` en LangGraph):

In [8]:
from typing import TypedDict

class AppState(TypedDict):
    """Estado compartido de la aplicación."""
    db_connection: object | None
    user_id: str
    session_data: dict[str, str]


def procesar_con_estado(state: AppState, query: str) -> list:
    """Procesa usando estado compartido (sin globales)."""
    db_connection: object | None = state.get("db_connection")
    if db_connection is None:
        return []
    return db_connection.execute(query)  # type: ignore

**🎓 Lección:** Pasar dependencias explícitamente hace el código testeable y predecible.

---

## 6️⃣ Manejo de Recursos con `finally`

Los recursos (archivos, conexiones, cursores) **SIEMPRE** deben cerrarse, incluso si hay errores.

In [9]:
# ❌ MAL: Sin garantía de cierre
def leer_archivo_mal(ruta: str) -> str:
    archivo = open(ruta, "r")
    contenido = archivo.read()
    archivo.close()  # ¿Se ejecuta si hay error en read()?
    return contenido


# ✅ BIEN: Con finally
def leer_archivo_bien(ruta: str) -> str:
    """Lee un archivo con manejo seguro de recursos.
    
    Args:
        ruta: Path al archivo
        
    Returns:
        Contenido del archivo
    """
    archivo = None
    contenido: str = ""
    
    try:
        archivo = open(ruta, "r")
        contenido = archivo.read()
    except FileNotFoundError:
        contenido = "Archivo no encontrado"
    finally:
        if archivo is not None:
            archivo.close()  # SIEMPRE se ejecuta
    
    return contenido


# ✅ MEJOR: Con context manager (with)
def leer_archivo_mejor(ruta: str) -> str:
    """Lee un archivo usando context manager."""
    try:
        with open(ruta, "r") as archivo:
            contenido: str = archivo.read()
            return contenido
    except FileNotFoundError:
        return "Archivo no encontrado"

### 🔌 Ejemplo con Base de Datos

In [10]:
def ejecutar_query_segura(connection: object, sql: str) -> list:
    """Ejecuta query con manejo seguro del cursor.
    
    Args:
        connection: Conexión a BD
        sql: Query a ejecutar
        
    Returns:
        Resultados de la query
    """
    cursor = None
    resultados: list = []
    
    try:
        cursor = connection.cursor()  # type: ignore
        cursor.execute(sql)
        resultados = cursor.fetchall()
    except Exception as e:
        print(f"Error en query: {e}")
        resultados = []
    finally:
        if cursor is not None:
            cursor.close()  # Garantizado incluso con error
    
    return resultados

**🎓 Lección:** `finally` garantiza limpieza de recursos. Para archivos, prefiere `with` (context manager).

---

## 7️⃣ Documentación Completa (Docstrings)

Todas las funciones públicas DEBEN tener docstrings con formato Google/NumPy/Sphinx.

In [11]:
# ❌ MAL: Sin documentación
def calcular(a, b, op):
    if op == "+":
        return a + b
    elif op == "-":
        return a - b
    return 0


# ✅ BIEN: Documentación completa (formato Google)
def calcular_operacion(a: float, b: float, operacion: str) -> float:
    """Realiza una operación matemática entre dos números.
    
    Args:
        a: Primer operando
        b: Segundo operando
        operacion: Operador matemático ('+', '-', '*', '/')
        
    Returns:
        Resultado de la operación. Retorna 0.0 si la operación no es válida.
        
    Raises:
        ZeroDivisionError: Si se intenta dividir por cero
        
    Examples:
        >>> calcular_operacion(10, 5, '+')
        15.0
        >>> calcular_operacion(10, 5, '-')
        5.0
    """
    resultado: float = 0.0
    
    if operacion == "+":
        resultado = a + b
    elif operacion == "-":
        resultado = a - b
    elif operacion == "*":
        resultado = a * b
    elif operacion == "/":
        if b == 0:
            raise ZeroDivisionError("No se puede dividir por cero")
        resultado = a / b
    
    return resultado

**🎓 Lección:** El código se lee más veces de las que se escribe. Invierte tiempo en documentar bien.

---

## 8️⃣ Validación con Excepciones Específicas

Captura excepciones específicas ANTES que genéricas. Esto permite manejar cada error apropiadamente.

In [ ]:
import json

# ❌ MAL: Solo Exception genérica
def parsear_json_mal(texto: str) -> dict:
    try:
        return json.loads(texto)
    except Exception as e:  # Demasiado genérico
        print(f"Error: {e}")
        return {}


# ✅ BIEN: Excepciones específicas primero
def parsear_json_bien(texto: str) -> dict[str, str]:
    """Parsea un texto JSON con manejo robusto de errores.
    
    Args:
        texto: String JSON a parsear
        
    Returns:
        Diccionario parseado o vacío si hay error
    """
    resultado: dict[str, str] = {}
    
    try:
        datos = json.loads(texto)
        resultado = datos
        
    except json.JSONDecodeError as e:
        # Error específico de JSON
        print(f"Error de formato JSON: {e}")
        resultado = {"error": "json_invalido"}
        
    except TypeError as e:
        # Error de tipo (ej: texto es None)
        print(f"Error de tipo: {e}")
        resultado = {"error": "tipo_invalido"}
        
    except Exception as e:
        # Otros errores inesperados (último recurso)
        print(f"Error inesperado: {e}")
        resultado = {"error": "error_desconocido"}
    
    return resultado

### 🎯 Orden de Captura (Más Específico → Más General)

In [ ]:
def operacion_db_compleja(connection: object, sql: str) -> str:
    """Ejemplo de manejo jerárquico de excepciones."""
    estado: str = "error_desconocido"
    
    try:
        cursor = connection.cursor()  # type: ignore
        cursor.execute(sql)
        cursor.close()
        estado = "exito"
        
    except AttributeError:
        # connection no tiene método cursor()
        estado = "conexion_invalida"
        
    except ConnectionError:
        # Problemas de red/conexión
        estado = "error_conexion"
        
    except ValueError:
        # SQL con valores inválidos
        estado = "sql_invalido"
        
    except Exception as e:
        # Último recurso
        print(f"Error inesperado: {e}")
        estado = "error_desconocido"
    
    return estado

**🎓 Lección:** 
- Capturar `Exception` solo debe ser el **último recurso**
- Siempre intenta capturar excepciones específicas primero
- Esto permite dar mensajes de error más útiles

---

## 9️⃣ Type Aliases para Tipos Complejos

Cuando un tipo se repite mucho o es complejo, créale un alias.

In [ ]:
from typing import TypeAlias, Literal

# ❌ MAL: Repetir tipos largos
def procesar1(estado: Literal["iniciar", "avanzar", "repetir", "finalizado"]) -> None:
    pass

def procesar2(estado: Literal["iniciar", "avanzar", "repetir", "finalizado"]) -> None:
    pass


# ✅ BIEN: Usar TypeAlias
EstadoFlujo: TypeAlias = Literal["iniciar", "avanzar", "repetir", "finalizado"]
SeñalDB: TypeAlias = Literal["correcto", "error", "sin_resultados"]
ConfiguracionDB: TypeAlias = dict[str, str | int]

def procesar_flujo(estado: EstadoFlujo) -> SeñalDB:
    """Procesa un flujo de estados."""
    if estado == "iniciar":
        return "correcto"
    return "error"

def inicializar_db(config: ConfiguracionDB) -> bool:
    """Inicializa base de datos con configuración."""
    return True

**🎓 Lección:** Type aliases mejoran legibilidad y mantenibilidad cuando cambias tipos.

---

## 🎯 Resumen: Checklist de Buenas Prácticas

Al escribir código Python profesional, asegúrate de cumplir con:

### ✅ Type System
- [ ] Type hints en **todas** las funciones (parámetros y retorno)
- [ ] Type hints en **todas** las variables (incluso locales)
- [ ] Usar sintaxis moderna: `str | None` en lugar de `Optional[str]`
- [ ] Usar `TypeAlias` para tipos complejos repetidos

### ✅ Control de Flujo
- [ ] **Máximo 2 returns** por función (1 al final + guard clauses opcionales)
- [ ] Guard clauses al inicio para validaciones
- [ ] Evitar anidamiento > 3 niveles

### ✅ Arquitectura
- [ ] Funciones pequeñas (< 30 líneas)
- [ ] Funciones auxiliares con prefijo `_`
- [ ] Evitar variables globales dentro de funciones
- [ ] Pasar dependencias como parámetros

### ✅ Manejo de Errores
- [ ] Capturar excepciones **específicas** antes que `Exception`
- [ ] Usar `finally` para liberar recursos
- [ ] Preferir `with` (context managers) para archivos

### ✅ Documentación
- [ ] Docstrings completos en funciones públicas
- [ ] Formato consistente (Google/NumPy/Sphinx)
- [ ] Incluir: Args, Returns, Raises, Examples

---

## 📖 Recursos Adicionales

- [PEP 8 – Style Guide for Python Code](https://peps.python.org/pep-0008/)
- [PEP 484 – Type Hints](https://peps.python.org/pep-0484/)
- [PEP 604 – Union Types (X | Y)](https://peps.python.org/pep-0604/)
- [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html)
- [Real Python - Type Checking](https://realpython.com/python-type-checking/)

---

## 🎓 Conclusión

**Python puede ser tan disciplinado como Java, C# o TypeScript** si seguimos estas reglas. El código resultante es:

- ✅ **Más legible**: Otros desarrolladores lo entienden rápidamente
- ✅ **Más mantenible**: Cambios futuros son más fáciles
- ✅ **Más robusto**: Errores se detectan antes
- ✅ **Más testeable**: Funciones pequeñas y puras son fáciles de testear
- ✅ **Más profesional**: Demuestra expertise y cuidado

**Estas prácticas se aplican en TODOS los notebooks de este repositorio.** 🚀

---

## 💡 Lección del Mundo Real: La Importancia de la Coherencia

### 📖 Historia Real de Este Proyecto

Durante el desarrollo de este repositorio, ocurrió algo **muy instructivo** que demuestra la importancia de mantener la documentación sincronizada con el código:

**El Problema:**
- El archivo `README.md` especificaba: "Python 3.8+"
- PERO todo el código usaba características de **Python 3.10+**:
  - Sintaxis `str | None` en lugar de `Optional[str]` (PEP 604)
  - `TypeAlias` y `Literal` por todas partes
  - Todos los notebooks tenían validaciones `assert current >= (3, 10, 0)`

**¿Qué hubiera pasado?**

Si un estudiante con Python 3.8 o 3.9 intentara ejecutar el código:

```python
# Python 3.8/3.9
nombre: str | None = None  # ❌ SyntaxError: invalid syntax
```

El programa fallaría **inmediatamente** con un error críptico. El estudiante pensaría:
- "El README dice 3.8+, ¿por qué no funciona?"
- "¿Hice algo mal en la instalación?"
- Pérdida de tiempo debugueando un problema que no es su culpa

**La Solución:**

Actualizamos el README para reflejar la realidad:
```markdown
## 🧰 Requisitos
- **Python 3.10+** (requerido para sintaxis moderna de tipos)
```

### 🎯 Lecciones para Programadores Profesionales

#### 1️⃣ **La Documentación es Código**

La documentación desactualizada es **tan grave como un bug en el código**. Puede:
- ❌ Frustrar a los usuarios
- ❌ Generar issues falsos
- ❌ Dañar la reputación del proyecto
- ❌ Desperdiciar tiempo de soporte

#### 2️⃣ **Valida Versiones Explícitamente**

**Siempre** añade validaciones al inicio de tu aplicación:

```python
import sys

required = (3, 10, 0)
current = sys.version_info[:3]

assert current >= required, (
    f"⚠️  Se requiere Python {required[0]}.{required[1]}.{required[2]} o superior. "
    f"Detectado: {current[0]}.{current[1]}.{current[2]}"
)
```

**Beneficios:**
- ✅ Error claro e inmediato
- ✅ Usuario sabe exactamente qué hacer
- ✅ Previene debugging innecesario

#### 3️⃣ **Sincroniza Requisitos con CI/CD**

En proyectos reales, usa herramientas que validen automáticamente:

```yaml
# .github/workflows/test.yml
jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: ["3.10", "3.11", "3.12"]  # ← Coherente con README
```

#### 4️⃣ **Checklist de Coherencia**

Antes de publicar cualquier proyecto, verifica:

- [ ] Versión de Python en `README.md`
- [ ] Versión en `pyproject.toml` o `setup.py`
- [ ] Versión en archivos `.github/workflows/*.yml`
- [ ] Validaciones en el código (`sys.version_info`)
- [ ] Características usadas vs versión mínima especificada

### 🏆 Aplica Esta Lección

La próxima vez que documentes un proyecto:
1. **Escribe los requisitos DESPUÉS de escribir el código**
2. **Valida explícitamente las versiones en runtime**
3. **Revisa la coherencia antes de cada commit**
4. **Automatiza estas validaciones con pre-commit hooks**

**Recuerda:** Un README incorrecto puede hacer más daño que no tener README. 📝✨